## Importing required libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing our list of stocks

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head(2)

,Ticker
0,A
1,AAL


## Acquiring an API Token

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making our first API call

 We need the following information from the API
-  Market capitalization for each stock
- Price of each stock

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 98930967, 'calculationPrice': 'close', 'change': 1.41, 'changePercent': 0.00844, 'close': 173.93, 'closeSource': 'iloiffca', 'closeTime': 1685985975474, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 174.412, 'delayedPriceTime': 1649556015615, 'extendedChange': 0.49, 'extendedChangePercent': 0.00286, 'extendedPrice': 178.98, 'extendedPriceTime': 1663889611760, 'high': 174.47, 'highSource': ' e nmieae1iupeddtyrlc5 ', 'highTime': 1702855294578, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 178.04, 'iexCloseTime': 1707230910544, 'iexLastUpdated': 1709740995717, 'iexMarketPercent': 0.01762690822165304, 'iexOpen': 168.39, 'iexOpenTime': 1718254369020, 'iexRealtimePrice': 172.14, 'iexRealtimeSize': 155, 'iexVolume': 1696026, 'lastTradeTime': 1691665770450, 'latestPrice': 178.09, 'latestSource': 'Close', 'latestTime': 'March 23, 2022', 'latestUpdate': 1727565786027, 'latestVolume': 102748663, 'low': 169.88, 'lowSource': ' 

## Parsing our API call

The API call that we executed in the last code block contains all the information required to build equal weight S&P 500 strategy


With that said, the data is not in a proper format yet and we need to parse it first.

In [17]:
price = data['latestPrice']
market_cap = data['marketCap']
print(price)
print('The market capitalization for Apple is:{} trillion'.format(round(market_cap/1000000000000, 2)))

172.67
The market capitalization for Apple is:2.84 trillion


## Adding all our stocks to a pandas dataframe

In [18]:
my_columns = ['Ticker', 'Stock_price', 'Market_capitalization', 'Number_of_shares_to_buy']
final_frame = pd.DataFrame([[symbol, price, market_cap, 'N/A']],columns = my_columns)
final_frame

,Ticker,Stock_price,Market_capitalization,Number_of_shares_to_buy
0,AAPL,172.67,2836388762325,N/A


In [13]:
# final_frame.append(pd.Series([symbol, price, market_cap, 'N/A'], index = my_columns)], ignore_index = True)

## Looping through the tickers in our list of stocks

In [14]:
# final_frame = pd.DataFrame(columns = my_columns)
# for symbol in stocks['Ticker']:
#     api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
#     data = requests.get(api_url).json()
#     final_frame = final_frame.append(
#                                         pd.Series([symbol, 
#                                                    data['latestPrice'], 
#                                                    data['marketCap'], 
#                                                    'N/A'], 
#                                                   index = my_columns), 
#                                         ignore_index = True)

# tickers = []
# for symbol in stocks['Ticker']:
#     try:
#         api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
#         data = requests.get(api_url).json()
#         print(data)
#         d = dict(zip(my_columns, [symbol, data['latestPrice'], data['marketCap'], 'N/A']))
#         tickers.append(d)
#     except:
#         continue
# final_frame = pd.DataFrame(tickers)


In [7]:
final_frame

,Ticker,Stock_price,Market_capitalization,Number_of_shares_to_buy
0,AAPL,169.71,2765086225116,N/A


## Using bach API calls to improve performance (instead of the above method)

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. 

Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [19]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [20]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
tickers = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            d = dict(zip(my_columns, [symbol,
                                      data[symbol]['quote']['latestPrice'],
                                      data[symbol]['quote']['marketCap'],
                                      'N/A']))
            tickers.append(d)
        except:
            continue

final_dataframe = pd.DataFrame(tickers)
final_dataframe 

,Ticker,Stock_price,Market_capitalization,Number_of_shares_to_buy
0,A,145.46,42604369020,N/A
1,AAL,16.91,10816406603,N/A
2,AAP,212.30,13025319937,N/A
3,AAPL,176.07,2851557417222,N/A
4,ABBV,167.44,294423482909,N/A
...,...,...,...,...
497,YUM,124.54,35395438396,N/A
498,ZBH,122.73,26004792920,N/A
499,ZBRA,434.00,24041847904,N/A
500,ZION,72.68,10827701393,N/A


## Calculating the portofolio size

In [21]:
portofolio_size = input('Please enter the value of your portofolio: ')
try:
    val = float(portofolio_size)
except ValueError:
    print('This is not a number! \nPlease try again')
    portofolio_size = input('Please enter the value of your portofolio: ')
    val = float(portofolio_size)

Please enter the value of your portofolio: 10000000


In [29]:
# position_size is a term to know the share of each stock (502) in the money that you invest 
position_size = val/len(final_dataframe.index)
print('the position size is: ${}'.format(position_size))


for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i , 'Number_of_shares_to_buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock_price'])

final_dataframe
    
# number_of_apple_shares = position_size/500
# print('the number of Apple share to buy are: {} share'.format(math.floor(number_of_apple_shares)))

the position size is: $19920.3187250996


,Ticker,Stock_price,Market_capitalization,Number_of_shares_to_buy
0,A,145.46,42604369020,136
1,AAL,16.91,10816406603,1178
2,AAP,212.30,13025319937,93
3,AAPL,176.07,2851557417222,113
4,ABBV,167.44,294423482909,118
...,...,...,...,...
497,YUM,124.54,35395438396,159
498,ZBH,122.73,26004792920,162
499,ZBRA,434.00,24041847904,45
500,ZION,72.68,10827701393,274


## Formatting Excel outputs

In [28]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'recommended_trades', index = False)

## Creating the Formats We'll Need For Our .xlsx File

Formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

- String format for tickers.
- XX.XX format for stock prices.
- XX.XXX format for market capitalization.
- Integer format for the number of shares to purchase.

In [30]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Applying the Formats to the Columns of Our .xlsx File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:


`writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column`

In [ ]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself".

Let's simplify this by putting it in 2 loops:

In [31]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['recommended_trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['recommended_trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output
Saving our Excel file is very easy:

In [32]:
writer.save()